# Imports

In [19]:
import numpy as np
import pandas as pd
from hmmlearn import hmm
import multiprocess as mp
from random import sample 

# Functions

In [11]:
# function for training 7-state HMM, fixed model structure
# inputs: data = dataframe of training pSTAT data, boot_rep = integer of numnber of bootstrapping reps to run
# default is no boostrapping, boot_samples = integer of number of training samples to use for bootstrapping runs 
# outputs: m = means of HMM, cov = covariance matrix of HMM, trans = transition matrix of HMM

def train_hmm(data,boot_reps=0,boot_samples=10000):
    # no bootstrapping
    if boot_reps == 0:
        # convergence check
        converge = False

        # setup 7-state Gaussian HMM model
        hmm_model = hmm.GaussianHMM(n_components=7, init_params='cm', params='cmt')

        # starting probabilities
        hmm_model.startprob_ = np.array([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]) # always start in the first state

        # initialize transition matrix 
        hmm_model.transmat_ = np.array([[1/3, 1/3, 1/3, 0, 0, 0, 0], 
                                        [0, 1/3, 0, 1/3, 1/3, 0, 0],
                                        [0, 0, 1/3, 1/3, 1/3, 0, 0],
                                        [0, 0, 0, 1/3, 0, 1/3, 1/3],
                                        [0, 0, 0, 0, 1/3, 1/3, 1/3],
                                        [0, 0, 0, 0, 0, 1, 0],
                                        [0, 0, 0, 0, 0, 0, 1]])

        # flatten entire training data
        train_flat = np.asarray(data).flatten().reshape(len(np.asarray(data).flatten()),1)
        train_length = [data.shape[1]] * data.shape[0]

        # run until training converges
        while converge == False:
            # train HMM
            hmm_model.fit(train_flat,train_length)

            # check convergence
            converge = hmm_model.monitor_.converged 

        # model specs to return
        m = np.squeeze(hmm_model.means_)
        cov = np.squeeze(hmm_model.covars_)
        trans = hmm_model.transmat_
    else:
        # add bootstrapped model specs
        m_sum = 0
        cov_sum = 0
        trans_sum = 0

        # train each bootstrap model
        for i in range(boot_reps):
            print(i)
            
            # convergence check
            converge = False

            # setup 7-state Gaussian HMM model
            hmm_model = hmm.GaussianHMM(n_components=7, init_params='cm', params='cmt')

            # starting probabilities
            hmm_model.startprob_ = np.array([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]) # always start in the first state

            # initialize transition matrix 
            hmm_model.transmat_ = np.array([[1/3, 1/3, 1/3, 0, 0, 0, 0], 
                                            [0, 1/3, 0, 1/3, 1/3, 0, 0],
                                            [0, 0, 1/3, 1/3, 1/3, 0, 0],
                                            [0, 0, 0, 1/3, 0, 1/3, 1/3],
                                            [0, 0, 0, 0, 1/3, 1/3, 1/3],
                                            [0, 0, 0, 0, 0, 1, 0],
                                            [0, 0, 0, 0, 0, 0, 1]])

            # bootstrap samples 
            ind = sample(list(np.arange(data.shape[0])),boot_samples)
            bootstrap_data = data.iloc[ind,:]

            # flatten data
            train_flat = np.asarray(bootstrap_data).flatten().reshape(len(np.asarray(bootstrap_data).flatten()),1)
            train_length = [bootstrap_data.shape[1]] * bootstrap_data.shape[0]

            # run until training converges
            while converge == False:
                # train HMM
                hmm_model.fit(train_flat,train_length)

                # check convergence
                converge = hmm_model.monitor_.converged 

            # add to model specs
            m_sum = m_sum + np.squeeze(hmm_model.means_)
            cov_sum = cov_sum + np.squeeze(hmm_model.covars_)
            trans_sum = trans_sum + hmm_model.transmat_

        # average model specs
        m = m_sum / boot_reps
        cov = cov_sum / boot_reps
        trans = trans_sum / boot_reps

    return m,cov,trans  

In [37]:
# function to detect state switches
# inputs: state_data = array of hmm state data from decoding step (samples x timepoints)
# outputs: mean_start_ind = array of mean start indices for timeframes, mean_end_ind = array of mean end indices for timeframes

def detect_swtiches(state_data):
    # place to store start and end ind, and state numbers
    start_ind = np.ones((state_data.shape[0],np.unique(state_data).shape[0])) * np.NaN
    end_ind = np.ones((state_data.shape[0],np.unique(state_data).shape[0])) * np.NaN

    # all start ind begin at 0
    start_ind[:,0] = 0

    # loop through each sample
    for i in range(state_data.shape[0]):
        check = state_data[i,0]
        count = 1 # position in start/end ind
        # loop through each time point
        for j in range(1,state_data.shape[1]):
            if check != state_data[i,j]:
                check = state_data[i,j]
                start_ind[i,count] = j
                end_ind[i,count-1] = j-1
                count+=1
                
        end_ind[i,count-1] = state_data.shape[1]-1
    
    # remove columns that have all nans
    start_ind = start_ind[:,~np.all(np.isnan(start_ind),axis=0)]
    end_ind = end_ind[:,~np.all(np.isnan(end_ind),axis=0)]

    # get mean position of switches
    mean_start_ind = np.nanmean(start_ind,axis=0)
    mean_end_ind = np.nanmean(end_ind,axis=0)

    return mean_start_ind,mean_end_ind

# Main Script

In [17]:
# load training data and labels (trajectories already normalized)
train_norm_pSTAT3_data_df = pd.read_csv('Data/subset_training_data_pSTAT3.csv', header=None)
train_label = np.asarray(pd.read_csv('Data/subset_training_label_pSTAT3.csv', header=None))

# load testing data and labels (trajectories already normalized)
test_norm_pSTAT3_data_df = pd.read_csv('Data/subset_testing_data_pSTAT3.csv', header=None)
test_label = np.asarray(pd.read_csv('Data/subset_testing_label_pSTAT3.csv', header=None))

In [ ]:
# train HMM with multiprocessing
# inputs for parallel training 
inputs1 = [
    (train_norm_pSTAT3_data_df,10,3000),
    (train_norm_pSTAT3_data_df,10,3000),
    (train_norm_pSTAT3_data_df,10,3000),
    (train_norm_pSTAT3_data_df,10,3000),
]

# pSTAT3 HMM
# results has means, covariances, and transitions of trained HMM for each bootstrapping run
with mp.Pool(4) as p:
    pSTAT3_results = p.starmap(train_hmm,inputs1)


In [33]:
# average bootstrapping runs
pSTAT3_m = (pSTAT3_results[0][0] + pSTAT3_results[1][0] + pSTAT3_results[2][0] + pSTAT3_results[3][0]) / 4
pSTAT3_cov = (pSTAT3_results[0][1] + pSTAT3_results[1][1] + pSTAT3_results[2][1] + pSTAT3_results[3][1]) / 4
pSTAT3_trans = (pSTAT3_results[0][2] + pSTAT3_results[1][2] + pSTAT3_results[2][2] + pSTAT3_results[3][2]) / 4

In [48]:
# use parameters from bootstrapping to identify hidden states of test set
# intialize pSTAT3 model 
pSTAT3_hmm = hmm.GaussianHMM(n_components=7, init_params='', params='') 

# set parameters
pSTAT3_hmm.startprob_ = np.array([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]) # always start in state 1
pSTAT3_hmm.means_ = pSTAT3_m.reshape(len(pSTAT3_m),1)
pSTAT3_hmm.covars_ = pSTAT3_cov.reshape(len(pSTAT3_cov),1)
pSTAT3_hmm.transmat_ = pSTAT3_trans

# flatten entire testing data (formatting appropriately)
pSTAT3_test_flat = np.asarray(test_norm_pSTAT3_data_df).flatten().reshape(len(np.asarray(test_norm_pSTAT3_data_df).flatten()),1)
pSTAT3_test_length = [test_norm_pSTAT3_data_df.shape[1]] * test_norm_pSTAT3_data_df.shape[0]

# fit hmm - not actually fitting/training because we set parameters
pSTAT3_hmm.fit(pSTAT3_test_flat,pSTAT3_test_length)

# decode hidden states for test data
pSTAT3_test_states = pSTAT3_hmm.decode(pSTAT3_test_flat,pSTAT3_test_length)[1]
pSTAT3_test_states = pSTAT3_test_states.reshape(test_norm_pSTAT3_data_df.shape[0],test_norm_pSTAT3_data_df.shape[1])

In [ ]:
# detect state switches across all trajectories
states = detect_swtiches(pSTAT3_test_states)